In [ ]:
!rm -rf `find -type d -name .ipynb_checkpoints`

In [1]:
import torch
from torch.utils.data import DataLoader
from PIL import Image
import json
import re
from typing import Optional
from codecarbon import EmissionsTracker
from os import listdir
from os.path import isfile, join
from tqdm.auto import tqdm
from PIL import Image
from tqdm.notebook import tqdm
from transformers import ViltConfig
from transformers import ViltProcessor
from transformers import ViltForQuestionAnswering
from carbontracker.tracker import CarbonTracker
import numpy as np
from dataloader import *
from train import *
torch.manual_seed(41)

In [2]:
config = ViltConfig.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
tr_image_root = '/home/ahuq/research/data/VQA_data/v1/images/train2014/'
tr_q_root = open('/home/ahuq/research/data/VQA_data/v1/questions/Questions_Train_mscoco/OpenEnded_mscoco_train2014_questions.json')
tr_a_root = open('/home/ahuq/research/data/VQA_data/v1/annotations/Annotations_Train_mscoco/mscoco_train2014_annotations.json')

val_image_root = '/home/ahuq/research/data/VQA_data/v1/images/val2014'
val_q_root = open('/home/ahuq/research/data/VQA_data/v1/questions/Questions_Val_mscoco/OpenEnded_mscoco_val2014_questions.json')
val_a_root = open('/home/ahuq/research/data/VQA_data/v1/annotations/Annotations_Val_mscoco/mscoco_val2014_annotations.json')

ts_image_root = '/home/ahuq/research/data/VQA_data/v1/images/val2014'
ts_q_root = open('/home/ahuq/research/data/VQA_data/v1/questions/Questions_Val_mscoco/OpenEnded_mscoco_val2014_questions.json')
ts_a_root = open('/home/ahuq/research/data/VQA_data/v1/annotations/Annotations_Val_mscoco/mscoco_val2014_annotations.json')

print("--------Training Data-------")
tr_questions, tr_annotations, tr_id_to_filename = load_data(tr_image_root,tr_q_root,tr_a_root, config)
print("--------Validation Data-------")
val_questions, val_annotations, val_id_to_filename = load_data(val_image_root,val_q_root,val_a_root, config)
print("--------Testing Data-------")
ts_questions, ts_annotations, ts_id_to_filename = load_data(ts_image_root,ts_q_root,ts_a_root, config)

--------Training Data-------


  0%|          | 0/82783 [00:00<?, ?it/s]

data questions keys: dict_keys(['info', 'task_type', 'data_type', 'license', 'data_subtype', 'questions'])
Number of questions: 248349
data annotations keys: dict_keys(['info', 'data_type', 'license', 'data_subtype', 'annotations'])
Number of annotations: 248349


  0%|          | 0/248349 [00:00<?, ?it/s]

--------Validation Data-------


...

  0%|          | 0/121512 [00:00<?, ?it/s]

--------Testing Data-------


  0%|          | 0/40504 [00:00<?, ?it/s]

data questions keys: dict_keys(['info', 'task_type', 'data_type', 'license', 'data_subtype', 'questions'])
Number of questions: 121512
data annotations keys: dict_keys(['info', 'data_type', 'license', 'data_subtype', 'annotations'])
Number of annotations: 121512


  0%|          | 0/121512 [00:00<?, ?it/s]

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-mlm")

tr_size = int(len(tr_questions)*0.1)
val_size = int(len(val_questions)*0.1)

tr_dataset = VQADataset(questions = tr_questions[:tr_size], 
                        annotations = tr_annotations[:tr_size], 
                        id_to_filename = tr_id_to_filename, 
                        processor = processor, 
                        config = config)
val_dataset = VQADataset(questions = val_questions[:val_size], 
                         annotations = val_annotations[:val_size], 
                         id_to_filename = val_id_to_filename, 
                         processor = processor,
                         config = config)
ts_dataset = VQADataset(questions = val_questions[val_size:val_size+1000], 
                        annotations = val_annotations[val_size:val_size+1000], 
                        id_to_filename = val_id_to_filename, 
                        processor = processor, 
                        config = config)


model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-mlm",id2label=config.id2label,label2id=config.label2id)
model = model.to(device)

Some weights of ViltForQuestionAnswering were not initialized from the model checkpoint at dandelin/vilt-b32-mlm and are newly initialized: ['classifier.0.bias', 'classifier.0.weight', 'classifier.1.bias', 'classifier.1.weight', 'classifier.3.bias', 'classifier.3.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
len(tr_dataset)

In [4]:
print("--------Training Data-------")

train_dataloader = DataLoader(tr_dataset, collate_fn=collate_fn, batch_size=32, shuffle=True)
tr_batch = next(iter(train_dataloader))
for k,v in tr_batch.items():
    print(k, v.shape)
    
print("--------Validation Data-------")
val_dataloader = DataLoader(val_dataset, collate_fn=collate_fn, batch_size=32, shuffle=True)
val_batch = next(iter(val_dataloader))
for k,v in val_batch.items():
    print(k, v.shape)

print("--------Testing Data-------")
test_dataloader = DataLoader(ts_dataset, collate_fn=collate_fn, batch_size=4, shuffle=True)
ts_batch = next(iter(test_dataloader))
for k,v in ts_batch.items():
    print(k, v.shape)

--------Training Data-------
input_ids torch.Size([32, 40])
attention_mask torch.Size([32, 40])
token_type_ids torch.Size([32, 40])
pixel_values torch.Size([32, 3, 576, 608])
pixel_mask torch.Size([32, 576, 608])
labels torch.Size([32, 3129])
--------Validation Data-------
input_ids torch.Size([32, 40])
attention_mask torch.Size([32, 40])
token_type_ids torch.Size([32, 40])
pixel_values torch.Size([32, 3, 576, 608])
pixel_mask torch.Size([32, 576, 608])
labels torch.Size([32, 3129])
--------Testing Data-------
input_ids torch.Size([4, 40])
attention_mask torch.Size([4, 40])
token_type_ids torch.Size([4, 40])
pixel_values torch.Size([4, 3, 576, 576])
pixel_mask torch.Size([4, 576, 576])
labels torch.Size([4, 3129])


In [ ]:
import torch
import torch.nn.functional as F
from tqdm import tqdm

# Optimizer setup
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
epochs = 100
model_name = 'teacher_0.3'
save_path = '/home/ahuq/research/CS791_LLM_Cleaned/models/'
tracker = EmissionsTracker()
tracker.start_task("teacher_0.3")
teacher_train_results = train_and_validate(model,train_dataloader, val_dataloader, optimizer, epochs,save_path, model_name, device)
step_emissions = tracker.stop_task("teacher_0.3")
# Stop tracking emissions for the task
emissions = tracker.stop()
print(f"Emissions for the step: {step_emissions} kg CO2")
print(f"Emissions for the task: {emissions} kg CO2")

In [5]:
#student model

sample = ViltConfig.from_pretrained("dandelin/vilt-b32-mlm",id2label=config.id2label, label2id=config.label2id)
sample.num_hidden_layers = 6  # Reduce number of transformer layers (e.g., half the original)
sample.hidden_size = 384   #768    # Reduce the hidden size (e.g., smaller than original 768)
student_model = ViltForQuestionAnswering(sample)
student_model = student_model.to(device)
# print(student_model)

In [ ]:
optimizer = torch.optim.AdamW(student_model.parameters(), lr=5e-5)
epochs = 100
model_name = 'student_few'
save_path = '/home/ahuq/research/CS791_LLM_Cleaned/student_models/'
tracker = EmissionsTracker()
tracker.start_task("student_0.3")
student_train_results = train_and_validate(student_model,train_dataloader, val_dataloader, optimizer, epochs,save_path, model_name, device)
step_emissions = tracker.stop_task("student_0.3")
# Stop tracking emissions for the task
emissions = tracker.stop()
print(f"Emissions for the step: {step_emissions} kg CO2")
print(f"Emissions for the task: {emissions} kg CO2")

In [ ]:
val_batch = next(iter(val_dataloader))
val_batch = {k: v.to(device) for k, v in val_batch.items()}

teacher_model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-mlm",id2label=config.id2label,label2id=config.label2id)  # Initialize with any required arguments
teacher_path = '/home/ahuq/research/CS791_LLM_Cleaned/models/teacher_0.3.pth'
teacher_model.load_state_dict(torch.load(teacher_path))

student_model = ViltForQuestionAnswering(sample)
student_model = student_model.to(device)
student_path = '/home/ahuq/research/CS791_LLM_Cleaned/student_models/student_few.pth'
student_model.load_state_dict(torch.load(student_path))

student_model= student_model.to(device)
teacher_model = teacher_model.to(device)

student_model.eval()
teacher_model.eval()

teacher_output = test(teacher_model, test_dataloader, device)
student_output = test(student_model, test_dataloader, device)
# with torch.no_grad():
#     student_outputs = student_model(**val_batch, output_hidden_states=True, output_attentions=True)
    # teacher_outputs = teacher_model(**val_batch, output_hidden_states=True, output_attentions=True)

In [6]:
### logits based train student model
sample = ViltConfig.from_pretrained("dandelin/vilt-b32-mlm",
                                                 id2label=config.id2label,
                                                 label2id=config.label2id)
sample.num_hidden_layers = 6  # Reduce number of transformer layers (e.g., half the original)
sample.hidden_size = 384   #768    # Reduce the hidden size (e.g., smaller than original 768)
student_model = ViltForQuestionAnswering(sample)

teacher_model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-mlm",id2label=config.id2label,label2id=config.label2id)  # Initialize with any required arguments
teacher_path = '/home/ahuq/research/CS791_LLM_Cleaned/models/teacher_0.3.pth'
teacher_model.load_state_dict(torch.load(teacher_path))

student_model= student_model.to(device)
teacher_model = teacher_model.to(device)
student_model.eval()
teacher_model.eval()

epochs = 100
alpha = 0.7
save_path = '/home/ahuq/research/CS791_LLM_Cleaned/student_models/'
optimizer = torch.optim.AdamW(student_model.parameters(), lr=5e-5)
tracker = EmissionsTracker()
tracker.start_task("student_logits_0.3")
KD_logit_results = KD_train_based_logits(teacher_model, student_model, train_dataloader, alpha, optimizer, epochs,save_path,device)
step_emissions = tracker.stop_task("student_logits_0.3")
# Stop tracking emissions for the task
emissions = tracker.stop()
print(f"Emissions for the step: {step_emissions} kg CO2")
print(f"Emissions for the task: {emissions} kg CO2")

Some weights of ViltForQuestionAnswering were not initialized from the model checkpoint at dandelin/vilt-b32-mlm and are newly initialized: ['classifier.0.bias', 'classifier.0.weight', 'classifier.1.bias', 'classifier.1.weight', 'classifier.3.bias', 'classifier.3.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_235270/2182275822.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless

Epoch: 0


  1%|▊                                                                              | 1/100 [13:03<21:32:13, 783.17s/it]

Validation - Loss: 91.9800, Top-5 Accuracy: 0.3321, Overall Accuracy: 0.1606
Epoch: 1


  2%|█▌                                                                             | 2/100 [26:05<21:18:21, 782.67s/it]

...

 99%|█████████████████████████████████████████████████████████████████████████████▏| 99/100 [21:23:09<12:53, 773.47s/it]

Validation - Loss: 0.4432, Top-5 Accuracy: 0.9846, Overall Accuracy: 0.5129
Epoch: 99


100%|█████████████████████████████████████████████████████████████████████████████| 100/100 [21:35:58<00:00, 777.58s/it]
[codecarbon WARNING @ 19:32:50] Background scheduler didn't run for a long period (77758s), results might be inaccurate
[codecarbon INFO @ 19:32:50] Energy consumed for RAM : 0.125670 kWh. RAM Power : 5.818180561065674 W
[codecarbon INFO @ 19:32:50] Energy consumed for all GPUs : 3.903968 kWh. Total GPU Power : 180.7431141739055 W
[codecarbon INFO @ 19:32:50] Energy consumed for all CPUs : 0.917981 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:32:50] 4.947619 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:32:50] Energy consumed for RAM : 0.125670 kWh. RAM Power : 5.818180561065674 W
[codecarbon INFO @ 19:32:50] Energy consumed for all GPUs : 3.903968 kWh. Total GPU Power : 0.0 W
[codecarbon INFO @ 19:32:50] Energy consumed for all CPUs : 0.917981 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:32:50] 4.947619 kWh of electricity used sinc

Validation - Loss: 0.4107, Top-5 Accuracy: 0.9847, Overall Accuracy: 0.5132
Logits states based Student model saved to /home/ahuq/research/CS791_LLM_Cleaned/student_models/
Emissions for the step: EmissionsData(timestamp='2024-11-28T19:32:50', project_name='codecarbon', run_id='32f73a92-622b-4087-81b9-e8ed0fd226dc', experiment_id='5b0fa12a-3dd7-45bb-9766-cc326314d9f1', duration=77758.34916458401, emissions=1.7278381455822254, emissions_rate=2.222061251899687e-05, cpu_power=42.5, gpu_power=180.7431141739055, ram_power=5.818180561065674, cpu_energy=0.9179805173971611, gpu_energy=3.903968226783192, ram_energy=0.1256700260736666, energy_consumed=4.947618770254019, country_name='United States', country_iso_code='USA', region='nevada', cloud_provider='', cloud_region='', os='Linux-5.15.167.4-microsoft-standard-WSL2-x86_64-with-glibc2.35', python_version='3.12.4', codecarbon_version='2.7.4', cpu_count=24, cpu_model='12th Gen Intel(R) Core(TM) i9-12900F', gpu_count=1, gpu_model='1 x Unknown GP

/home/ahuq/anaconda3/lib/python3.12/site-packages/codecarbon/output_methods/file.py:81: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(


In [7]:
### hidden states KD student training 
sample = ViltConfig.from_pretrained("dandelin/vilt-b32-mlm",
                                                 id2label=config.id2label,
                                                 label2id=config.label2id)
sample.num_hidden_layers = 6  # Reduce number of transformer layers (e.g., half the original)
sample.hidden_size = 384   #768    # Reduce the hidden size (e.g., smaller than original 768)
student_model = ViltForQuestionAnswering(sample)

teacher_model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-mlm",id2label=config.id2label,label2id=config.label2id)  # Initialize with any required arguments
teacher_path = '/home/ahuq/research/CS791_LLM_Cleaned/models/teacher_0.3.pth'
teacher_model.load_state_dict(torch.load(teacher_path))

student_model= student_model.to(device)
teacher_model = teacher_model.to(device)
# student_model.eval()
teacher_model.eval()

epochs = 100
alpha = 0.7
save_path = '/home/ahuq/research/CS791_LLM_Cleaned/student_models/'
# Instantiate the projector (e.g., teacher 768 -> student 384)
projector = HiddenStateProjector(teacher_dim=768, student_dim=384).to('cpu')
optimizer = torch.optim.AdamW(student_model.parameters(), lr=5e-5)
tracker = EmissionsTracker()
tracker.start_task("student_hidden_0.3")
KD_hidden_results = KD_train_based_hidden_states(teacher_model, student_model, train_dataloader, projector, alpha, optimizer, epochs,save_path,device)
step_emissions = tracker.stop_task("student_hidden_0.3")
# Stop tracking emissions for the task
emissions = tracker.stop()
print(f"Emissions for the step: {step_emissions} kg CO2")
print(f"Emissions for the task: {emissions} kg CO2")

Some weights of ViltForQuestionAnswering were not initialized from the model checkpoint at dandelin/vilt-b32-mlm and are newly initialized: ['classifier.0.bias', 'classifier.0.weight', 'classifier.1.bias', 'classifier.1.weight', 'classifier.3.bias', 'classifier.3.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_235270/2859843511.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless

Epoch: 0


  1%|▊                                                                              | 1/100 [13:42<22:37:34, 822.78s/it]

Validation - Loss: 70.1267, Top-5 Accuracy: 0.2726, Overall Accuracy: 0.1416
Epoch: 1


  2%|█▌                                                                             | 2/100 [27:05<22:04:27, 810.89s/it]

...

 99%|█████████████████████████████████████████████████████████████████████████████▏| 99/100 [22:20:19<13:31, 811.53s/it]

Validation - Loss: 0.4041, Top-5 Accuracy: 0.9847, Overall Accuracy: 0.5134
Epoch: 99


100%|█████████████████████████████████████████████████████████████████████████████| 100/100 [22:33:46<00:00, 812.27s/it]
[codecarbon WARNING @ 18:42:27] Background scheduler didn't run for a long period (81227s), results might be inaccurate
[codecarbon INFO @ 18:42:27] Energy consumed for RAM : 0.131276 kWh. RAM Power : 5.818180561065674 W
[codecarbon INFO @ 18:42:27] Energy consumed for all GPUs : 4.057545 kWh. Total GPU Power : 179.83125484341127 W
[codecarbon INFO @ 18:42:27] Energy consumed for all CPUs : 0.958930 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:42:27] 5.147751 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:42:27] Energy consumed for RAM : 0.131276 kWh. RAM Power : 5.818180561065674 W
[codecarbon INFO @ 18:42:27] Energy consumed for all GPUs : 4.057545 kWh. Total GPU Power : 0.0 W
[codecarbon INFO @ 18:42:27] Energy consumed for all CPUs : 0.958930 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:42:27] 5.147751 kWh of electricity used sin

Validation - Loss: 0.4052, Top-5 Accuracy: 0.9846, Overall Accuracy: 0.5134
Hidden states based Student model saved to /home/ahuq/research/CS791_LLM_Cleaned/student_models/
Emissions for the step: EmissionsData(timestamp='2024-11-30T18:42:27', project_name='codecarbon', run_id='cea11da1-ed0e-4b70-875a-44a816d447ff', experiment_id='5b0fa12a-3dd7-45bb-9766-cc326314d9f1', duration=81227.03557761502, emissions=1.7977295082599491, emissions_rate=2.2132156753948547e-05, cpu_power=42.5, gpu_power=179.83125484341127, ram_power=5.818180561065674, cpu_energy=0.9589303338132884, gpu_energy=4.057544510477456, ram_energy=0.13127599200607828, energy_consumed=5.147750836296823, country_name='United States', country_iso_code='USA', region='nevada', cloud_provider='', cloud_region='', os='Linux-5.15.167.4-microsoft-standard-WSL2-x86_64-with-glibc2.35', python_version='3.12.4', codecarbon_version='2.7.4', cpu_count=24, cpu_model='12th Gen Intel(R) Core(TM) i9-12900F', gpu_count=1, gpu_model='1 x Unknown

/home/ahuq/anaconda3/lib/python3.12/site-packages/codecarbon/output_methods/file.py:81: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(


In [ ]:
#student training attention based

sample = ViltConfig.from_pretrained("dandelin/vilt-b32-mlm",
                                                 id2label=config.id2label,
                                                 label2id=config.label2id)
sample.num_hidden_layers = 6  # Reduce number of transformer layers (e.g., half the original)
sample.hidden_size = 384   #768    # Reduce the hidden size (e.g., smaller than original 768)
student_model = ViltForQuestionAnswering(sample)

teacher_model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-mlm",id2label=config.id2label,label2id=config.label2id)  # Initialize with any required arguments
teacher_path = '/home/ahuq/research/CS791_LLM_Cleaned/models/teacher_0.3.pth'
teacher_model.load_state_dict(torch.load(teacher_path))

student_model= student_model.to(device)
teacher_model = teacher_model.to(device)
student_model.eval()
teacher_model.eval()

epochs = 100
alpha = 0.7
mapping_strategy = 'direct'
save_path = '/home/ahuq/research/CS791_LLM_Cleaned/student_models/'
optimizer = torch.optim.AdamW(student_model.parameters(), lr=5e-5)
tracker = EmissionsTracker()
tracker.start_task("student_attention_0.3")
KD_attention_results = KD_train_based_attentions(teacher_model, student_model, train_dataloader, projector, alpha, mapping_strategy, optimizer, epochs,save_path, device)
step_emissions = tracker.stop_task("student_attention_0.3")
# Stop tracking emissions for the task
emissions = tracker.stop()
print(f"Emissions for the step: {step_emissions} kg CO2")
print(f"Emissions for the task: {emissions} kg CO2")

Some weights of ViltForQuestionAnswering were not initialized from the model checkpoint at dandelin/vilt-b32-mlm and are newly initialized: ['classifier.0.bias', 'classifier.0.weight', 'classifier.1.bias', 'classifier.1.weight', 'classifier.3.bias', 'classifier.3.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_235270/2058725346.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless

Epoch: 0


  1%|▊                                                                              | 1/100 [12:21<20:23:03, 741.24s/it]

Validation - Loss: 69.7895, Top-5 Accuracy: 0.2739, Overall Accuracy: 0.1390
Epoch: 1


  2%|█▌                                                                             | 2/100 [24:42<20:10:32, 741.15s/it]

...

Validation - Loss: 0.2075, Top-5 Accuracy: 0.9844, Overall Accuracy: 0.5127
Epoch: 84


 85%|████████████████████████████████████████████████████████████████▌           | 85/100 [17:36:09<3:07:16, 749.10s/it]

Validation - Loss: 0.2025, Top-5 Accuracy: 0.9846, Overall Accuracy: 0.5130
Epoch: 85


 86%|█████████████████████████████████████████████████████████████████▎          | 86/100 [17:48:41<2:55:00, 750.00s/it]

Validation - Loss: 0.1989, Top-5 Accuracy: 0.9847, Overall Accuracy: 0.5132
Epoch: 86


In [ ]:
#Hybrid KD
sample = ViltConfig.from_pretrained("dandelin/vilt-b32-mlm",
                                                 id2label=config.id2label,
                                                 label2id=config.label2id)
sample.num_hidden_layers = 6  # Reduce number of transformer layers (e.g., half the original)
sample.hidden_size = 384   #768    # Reduce the hidden size (e.g., smaller than original 768)
student_model = ViltForQuestionAnswering(sample)

teacher_model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-mlm",id2label=config.id2label,label2id=config.label2id)  # Initialize with any required arguments
teacher_path = '/home/ahuq/research/CS791_LLM_Cleaned/models/teacher_0.3.pth'
teacher_model.load_state_dict(torch.load(teacher_path))

student_model= student_model.to(device)
teacher_model = teacher_model.to(device)
student_model.eval()
teacher_model.eval()

epochs = 100
alpha = 0.7
mapping_strategy = 'direct'
save_path = '/home/ahuq/research/CS791_LLM_Cleaned/student_models/'
optimizer = torch.optim.AdamW(student_model.parameters(), lr=5e-5)
projector = HiddenStateProjector(teacher_dim=768, student_dim=384).to('cpu')

tracker = EmissionsTracker()
tracker.start_task("student_hybrid_0.3")
# Your training code here

KD_hybrid_results = KD_train_based_hybrid(teacher_model, student_model, train_dataloader, projector, alpha, mapping_strategy, optimizer, epochs, save_path, device)
step_emissions = tracker.stop_task("student_hybrid_0.3")
# Stop tracking emissions for the task
emissions = tracker.stop()
print(f"Emissions for the step: {step_emissions} kg CO2")
print(f"Emissions for the task: {emissions} kg CO2")

In [ ]:
print(emissions)

In [ ]:
from codecarbon import EmissionsTracker

In [ ]:
from carbontracker.tracker import CarbonTracker

In [ ]:
!pip install --upgrade codecarbon

In [ ]:
from codecarbon import EmissionsTracker

# Initialize the emissions tracker
tracker = EmissionsTracker()

# Start tracking emissions for a task
tracker.start()

# Simulate a task
tracker.start_task("test")
for i in range(1000000):
    pass 
step_emissions = tracker.stop_task("test")

# Stop tracking emissions for the task
emissions = tracker.stop()

# Print the emissions data
print(f"Emissions for the step: {step_emissions} kg CO2")
print(f"Emissions for the task: {emissions} kg CO2")